In [280]:
import pandas as pd

df_participacion_2023_1 = pd.read_csv("participacion_descripcion_2023_1.csv")
df_participacion_2023_2 = pd.read_csv("participacion_descripcion_2023_2.csv")
df_participacion_2024 = pd.read_csv("participacion_descripcion_2024.csv")
df_parlamentarios = pd.read_csv("parlamentarios_info_general.csv")
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from nltk.corpus import stopwords
import nltk


In [303]:
# Parte del codigo encargada de contener lo necesario para sacar la keywords de los textos
# Descargar si no lo has hecho antes
nltk.download('stopwords')

# Usamos las stopwords en español
stopwords_es = stopwords.words('spanish')

def obtener_tema_con_lda(textos, n_temas=1):
    if isinstance(textos, str):
        textos = [textos]  # Si es un solo texto, lo convertimos a lista
    
    vectorizer = CountVectorizer(stop_words=stopwords_es)
    X = vectorizer.fit_transform(textos)
    
    lda = LatentDirichletAllocation(n_components=n_temas, random_state=0)
    lda.fit(X)

    palabras = vectorizer.get_feature_names_out()
    temas = []
    for topic in lda.components_:
        top_palabras = [palabras[i] for i in topic.argsort()[-5:]]
        temas.append(", ".join(top_palabras))
    
    return temas[0]  # Retorna el tema principal (solo el primer tema)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sebastián\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [304]:
df_participacion_2023_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24050 entries, 0 to 24049
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID_PARTICIPACION    24050 non-null  int64  
 1   PARLAMENTARIO_ID    24050 non-null  int64  
 2   FECHA               24050 non-null  object 
 3   LEGISLATURA         24050 non-null  int64  
 4   SESION              24050 non-null  int64  
 5   TIPO_SESION         24050 non-null  object 
 6   TIPO_PARTICIPACION  24050 non-null  object 
 7   CAMARA              24050 non-null  object 
 8   PARLAMENTARIOS      24050 non-null  object 
 9   TEXTO_PRINCIPAL     24050 non-null  object 
 10  TEXTO_ANTECEDENTE   12811 non-null  object 
 11  DESCRIPCION_DEBATE  24050 non-null  object 
 12  BOLETIN_ID          8892 non-null   float64
dtypes: float64(1), int64(4), object(8)
memory usage: 2.4+ MB


In [305]:
df_participation_2023_1.drop_duplicates()

,ID_PARTICIPACION,PARLAMENTARIO_ID,FECHA,LEGISLATURA,SESION,TIPO_SESION,TIPO_PARTICIPACION,CAMARA,PARLAMENTARIOS,TEXTO_PRINCIPAL,TEXTO_ANTECEDENTE,DESCRIPCION_DEBATE,BOLETIN_ID
0,391,3494,2023-06-19,371,31,Sesión parlamentaria especial,Intervención,Senado,[3494],"La señora ALLENDE.- \nGracias, Presidente. \nP...",El señor GUZMAN ( Secretario general ).- \nGra...,INQUIETUDES RELATIVAS A PLAN DE INVERSIONES EN...,NaN
1,392,3494,2023-06-19,371,32,Sesión parlamentaria extraordinaria,Intervención,Senado,[3494],"La señora ALLENDE.- \nGracias, Presidente. \nE...",NaN,CREACIÓN DE MINISTERIO DE SEGURIDAD PÚBLICA,NaN
2,393,3494,2023-06-14,371,30,Sesión parlamentaria ordinaria,Proyecto de Acuerdo,Senado,"[4513, 4577, 3494, 5196, 3735, 4591, 3099, 461...",-Proyecto de Acuerdo de los Honorables Senador...,NaN,-Proyecto de Acuerdo de los Honorables Senador...,NaN
3,394,3494,2023-06-13,371,29,Sesión parlamentaria ordinaria,Petición de oficio,Senado,[3494],De la señora ALLENDE:\n \n\n \n ...,NaN,PETICIONES DE OFICIOS,NaN
4,395,3494,2023-06-07,371,28,Sesión parlamentaria ordinaria,Proyecto de Acuerdo,Senado,"[2128, 4577, 3494, 5196, 3399, 3735, 4591, 309...",Proyecto de Acuerdo de los Honorables Senadore...,NaN,Proyecto de Acuerdo de los Honorables Senadore...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24045,243070,4622,2023-01-04,370,112,Sesión parlamentaria ordinaria,Intervención,Cámara de Diputados,[4622],El señor MIROSEVIC (Presidente).- \nTiene la...,NaN,CUENTA,NaN
24046,243071,4622,2023-01-03,370,111,Sesión parlamentaria ordinaria,Proyecto de Resolución,Cámara de Diputados,"[5239, 5261, 5264, 5219, 4583, 5190, 4602, 455...",El señor MIROSEVIC (Presidente).- \n \nCorresp...,NaN,ADOPCIÓN DE MEDIDAS PARA ELIMINACIÓN GRADUAL D...,NaN
24047,243072,4622,2023-01-03,370,112,Sesión parlamentaria ordinaria,Intervención en Comisión,Cámara de Diputados,[4622],La diputada Yeomans pregunta si depende totalm...,Tratado y acordado en la sesión ordinaria cele...,﻿INFORME DE LA COMISION DE HACIENDA RECAÍDO EN...,15625.0
24048,243073,4622,2023-01-03,370,112,Sesión parlamentaria ordinaria,Intervención en Comisión,Cámara de Diputados,[4622],La diputada Yeomans preguntó cómo se resolverá...,Tratado y acordado en la sesión especial celeb...,﻿INFORME DE LA COMISION DE HACIENDA RECAÍDO EN...,15511.0


In [306]:
df_parlamentarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1199 entries, 0 to 1198
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PARLAMENTARIO_ID       1199 non-null   int64  
 1   NOMBRE_COMPLETO        1199 non-null   object 
 2   VIGENTE                1199 non-null   int64  
 3   FECHA_NACIMIENTO       206 non-null    object 
 4   CAMARA_ACTUAL          206 non-null    object 
 5   CAMARA                 206 non-null    object 
 6   INICIO                 206 non-null    float64
 7   FINAL                  206 non-null    float64
 8   PARTIDO_POLITICO       155 non-null    object 
 9   UNIDAD_QUE_REPRESENTA  205 non-null    object 
dtypes: float64(2), int64(2), object(6)
memory usage: 93.8+ KB


In [307]:
df_parlamentarios.drop_duplicates()

,PARLAMENTARIO_ID,NOMBRE_COMPLETO,VIGENTE,FECHA_NACIMIENTO,CAMARA_ACTUAL,CAMARA,INICIO,FINAL,PARTIDO_POLITICO,UNIDAD_QUE_REPRESENTA
0,4611,Cristhian Moreira Barros,1,1961-09-05,C.Diputados,C. Diputados,2022.0,2026.0,Partido Unión Demócrata Independiente,13er Distrito
1,5209,Daniela Serrano Salazar,1,1995-03-17,C.Diputados,C. Diputados,2022.0,2026.0,Partido Comunista de Chile,12° Distrito
2,5194,Cristian Tapia Ramos,1,1968-07-14,C.Diputados,C. Diputados,2022.0,2026.0,NaN,4° Distrito
3,5203,Cristian Araya Lerdo De Tejada,1,1988-09-30,C.Diputados,C. Diputados,2022.0,2026.0,Partido Republicano de Chile,11° Distrito
4,4647,Cristian Labbe Martinez,1,1980-03-06,C.Diputados,C. Diputados,2022.0,2026.0,Unión Demócrata Independiente,8º Distrito
...,...,...,...,...,...,...,...,...,...,...
1194,2007,Hernan Modesto Olave Verdugo,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1195,278,Hernan Rojo Avendano,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1196,1185,Hernan Vodanovic Schnake,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1197,205,Homero Gutierrez Roman,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Vemos que las columnas REDES_SOCIALES y SITIOS_WEB en especial no nos entregan nada de información sobre los parlamentarios, por lo que decido eliminar estas columnas

In [308]:
df_parlamentarios = df_parlamentarios[['PARLAMENTARIO_ID', 'NOMBRE_COMPLETO', 'VIGENTE', 'FECHA_NACIMIENTO', 'CAMARA_ACTUAL','CAMARA', 'INICIO','FINAL','PARTIDO_POLITICO','UNIDAD_QUE_REPRESENTA']]
df_parlamentarios

,PARLAMENTARIO_ID,NOMBRE_COMPLETO,VIGENTE,FECHA_NACIMIENTO,CAMARA_ACTUAL,CAMARA,INICIO,FINAL,PARTIDO_POLITICO,UNIDAD_QUE_REPRESENTA
0,4611,Cristhian Moreira Barros,1,1961-09-05,C.Diputados,C. Diputados,2022.0,2026.0,Partido Unión Demócrata Independiente,13er Distrito
1,5209,Daniela Serrano Salazar,1,1995-03-17,C.Diputados,C. Diputados,2022.0,2026.0,Partido Comunista de Chile,12° Distrito
2,5194,Cristian Tapia Ramos,1,1968-07-14,C.Diputados,C. Diputados,2022.0,2026.0,NaN,4° Distrito
3,5203,Cristian Araya Lerdo De Tejada,1,1988-09-30,C.Diputados,C. Diputados,2022.0,2026.0,Partido Republicano de Chile,11° Distrito
4,4647,Cristian Labbe Martinez,1,1980-03-06,C.Diputados,C. Diputados,2022.0,2026.0,Unión Demócrata Independiente,8º Distrito
...,...,...,...,...,...,...,...,...,...,...
1194,2007,Hernan Modesto Olave Verdugo,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1195,278,Hernan Rojo Avendano,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1196,1185,Hernan Vodanovic Schnake,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1197,205,Homero Gutierrez Roman,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [309]:
df_participacion_2023_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22240 entries, 0 to 22239
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID_PARTICIPACION    22240 non-null  int64  
 1   PARLAMENTARIO_ID    22240 non-null  int64  
 2   FECHA               22240 non-null  object 
 3   LEGISLATURA         22240 non-null  int64  
 4   SESION              22240 non-null  int64  
 5   TIPO_SESION         22240 non-null  object 
 6   TIPO_PARTICIPACION  22240 non-null  object 
 7   CAMARA              22240 non-null  object 
 8   PARLAMENTARIOS      22240 non-null  object 
 9   TEXTO_PRINCIPAL     22240 non-null  object 
 10  TEXTO_ANTECEDENTE   10509 non-null  object 
 11  DESCRIPCION_DEBATE  22240 non-null  object 
 12  BOLETIN_ID          6393 non-null   float64
dtypes: float64(1), int64(4), object(8)
memory usage: 2.2+ MB


In [310]:
df_participacion_2023_2.drop_duplicates()

,ID_PARTICIPACION,PARLAMENTARIO_ID,FECHA,LEGISLATURA,SESION,TIPO_SESION,TIPO_PARTICIPACION,CAMARA,PARLAMENTARIOS,TEXTO_PRINCIPAL,TEXTO_ANTECEDENTE,DESCRIPCION_DEBATE,BOLETIN_ID
0,217,3494,2023-12-22,371,89,Sesión parlamentaria ordinaria,Intervención en Comisión,Senado,[3494],"La indicación 14, del Presidente de la Repúbli...",Acordado en sesión celebrada el 18 de marzo de...,﻿INFORME DE LA COMISIÓN DE LA MUJER Y EQUIDAD ...,7567.0
1,218,3494,2023-12-22,371,89,Sesión parlamentaria ordinaria,Intervención en Comisión,Senado,[3494],La Senadora señora Allende consultó acerca de ...,Acordado en sesión celebrada el 18 de marzo de...,﻿INFORME DE LA COMISIÓN DE LA MUJER Y EQUIDAD ...,7567.0
2,219,3494,2023-12-22,371,89,Sesión parlamentaria ordinaria,Intervención en Comisión,Senado,[3494],La Senadora señora Allende consultó las razone...,Acordado en sesión celebrada el 18 de marzo de...,﻿INFORME DE LA COMISIÓN DE LA MUJER Y EQUIDAD ...,7567.0
3,220,3494,2023-12-22,371,89,Sesión parlamentaria ordinaria,Intervención en Comisión,Senado,[3494],La Senadora señora Allende consultó respecto d...,Acordado en sesión celebrada el 18 de marzo de...,﻿INFORME DE LA COMISIÓN DE LA MUJER Y EQUIDAD ...,7567.0
4,221,3494,2023-12-22,371,89,Sesión parlamentaria ordinaria,Intervención en Comisión,Senado,[3494],La Senadora señora Allende manifestó su discre...,Acordado en sesión celebrada el 18 de marzo de...,﻿INFORME DE LA COMISIÓN DE LA MUJER Y EQUIDAD ...,7567.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22235,242973,4622,2023-07-25,371,56,Sesión parlamentaria ordinaria,Petición de oficio,Cámara de Diputados,[4622],"Diputada Yeomans, doña Gael . Mesa coordinador...",NaN,PETICIONES DE OFICIO. ARTÍCULOS 9° Y 9° A DE L...,NaN
22236,242974,4622,2023-07-24,371,55,Sesión parlamentaria ordinaria,Intervención,Cámara de Diputados,[4622],El señor CIFUENTES (Presidente).- \n Tiene la...,El señor CIFUENTES (Presidente).- \n Correspo...,PROHIBICIÓN DE CONDICIONAMIENTO DE RENDICIÓN D...,NaN
22237,242975,4622,2023-07-05,371,51,Sesión parlamentaria ordinaria,Integración,Cámara de Diputados,"[4565, 4604, 4560, 4564, 4559, 5190, 5224, 462...","Finalmente, propongo integrar la Comisión Espe...",NaN,INTEGRACIÓN DE COMISIONES ESPECIALES INVESTIGA...,NaN
22238,242976,4622,2023-07-04,371,50,Sesión parlamentaria especial,Mociones,Cámara de Diputados,"[4570, 4571, 5200, 3669, 1782, 4622, 4598, 4600]",Proyecto iniciado en moción de los diputados s...,NaN,Proyecto iniciado en moción de los diputados s...,NaN


In [311]:
df_participacion_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21871 entries, 0 to 21870
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID_PARTICIPACION    21871 non-null  int64  
 1   PARLAMENTARIO_ID    21871 non-null  int64  
 2   FECHA               21871 non-null  object 
 3   LEGISLATURA         21871 non-null  int64  
 4   SESION              21871 non-null  int64  
 5   TIPO_SESION         21871 non-null  object 
 6   TIPO_PARTICIPACION  21871 non-null  object 
 7   CAMARA              21871 non-null  object 
 8   PARLAMENTARIOS      21871 non-null  object 
 9   TEXTO_PRINCIPAL     21871 non-null  object 
 10  TEXTO_ANTECEDENTE   9073 non-null   object 
 11  DESCRIPCION_DEBATE  21871 non-null  object 
 12  BOLETIN_ID          5308 non-null   float64
dtypes: float64(1), int64(4), object(8)
memory usage: 2.2+ MB


In [312]:
df_participacion_2024.drop_duplicates()

,ID_PARTICIPACION,PARLAMENTARIO_ID,FECHA,LEGISLATURA,SESION,TIPO_SESION,TIPO_PARTICIPACION,CAMARA,PARLAMENTARIOS,TEXTO_PRINCIPAL,TEXTO_ANTECEDENTE,DESCRIPCION_DEBATE,BOLETIN_ID
0,0,3494,2024-09-03,372,51,Sesión parlamentaria ordinaria,Intervención,Senado,[3494],"La señora ALLENDE.- \nGracias, Presidente. \nH...",El señor GARCÍA ( Presidente ).- \nEn el prime...,ACUERDO ENTRE CHILE Y ESTADOS UNIDOS DE AMÉRIC...,NaN
1,1,3494,2024-09-03,372,51,Sesión parlamentaria ordinaria,Intervención,Senado,[3494],"La señora ALLENDE.- \nGracias, Presidente. \nC...",El señor GARCÍA ( Presidente ).- \nProyecto de...,REGLAS DE IGUALDAD DE GÉNERO PARA DESIGNACIÓN ...,NaN
2,2,3494,2024-09-03,372,51,Sesión parlamentaria ordinaria,Intervención,Senado,[3494],"La señora ALLENDE.- \nGracias, Presidente. \nP...","El señor GARCÍA ( Presidente ).- \nProyecto, i...",IMPOSIBILIDAD DE ADJUDICACIÓN DE PROGRAMAS O I...,NaN
3,3,3494,2024-09-03,372,51,Sesión parlamentaria ordinaria,Intervención,Senado,[3494],"La señora ALLENDE.- \nPresidente , yo le pedí ...",El señor GARCÍA ( Presidente ).- \nEn el prime...,ACUERDO ENTRE CHILE Y ESTADOS UNIDOS DE AMÉRIC...,NaN
4,4,3494,2024-08-28,372,48,Sesión parlamentaria extraordinaria,Intervención,Senado,[3494],"La señora ALLENDE.- \nPresidente, como ha seña...",El señor GARCÍA ( Presidente ).- \nProyecto de...,MODIFICACIÓN DE CÓDIGO DE AGUAS EN LO RELATIVO...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21866,242897,4622,2024-01-09,371,128,Sesión parlamentaria ordinaria,Solicitud de Resolución,Cámara de Diputados,"[5239, 5261, 5219, 5190, 4602, 4551, 4622, 524...",El señor CIFUENTES (Presidente).- \n \n Corres...,NaN,PREOCUPACIÓN POR AFECTACIÓN A PATRIMONIO MUNIC...,NaN
21867,242898,4622,2024-01-08,371,128,Sesión parlamentaria ordinaria,Intervención en Comisión,Cámara de Diputados,[4622],La Diputada Yeomans (Presidenta) reconoció la ...,Tratado y acordado en la sesión especial celeb...,﻿INFORME DE LA COMISION DE HACIENDA RECAÍDO EN...,16498.0
21868,242899,4622,2024-01-03,371,124,Sesión parlamentaria ordinaria,Petición de oficio,Cámara de Diputados,[4622],"-DiputadaYeomans , doña Gael . Acciones dest...",NaN,OTROS DOCUMENTOS DE LA CUENTA,NaN
21869,242900,4622,2024-01-03,371,124,Sesión parlamentaria ordinaria,Petición de oficio,Cámara de Diputados,[4622],"-DiputadaYeomans , doña Gael . Acciones dest...",NaN,OTROS DOCUMENTOS DE LA CUENTA,NaN


In [316]:
texto = [df_participacion_2024.loc[21870, 'TEXTO_PRINCIPAL']]
texto


['El señor CIFUENTES (Presidente).- \\n \\n Corresponde tratar el proyecto de resolución N° 904. \\nEl señor Prosecretario dará lectura a su parte dispositiva. \\n \\nEl señor ROJAS (Prosecretario).- \\n \\nProyecto de resolución N° 904, de las diputadas  Mercedes Bulnes ,   María Francisca Bello ,   Marcela Riquelme  y   Gael Yeomans , y el diputado  Gonzalo Winter , que en su parte dispositiva señala: \\nLa Cámara de Diputados solicita a su excelencia el Presidente de la República que presente un proyecto para modificar la ley Nº 19.496, dándole la facultad al Servicio Nacional del Consumidor para tramitar y resolver procedimientos sancionatorios y dictar normas e instrucciones de carácter general de naturaleza obligatoria. \\n']

In [313]:


df_participacion_2024["KEYWORDS"] = df_participacion_2024["TEXTO_PRINCIPAL"].apply(obtener_tema_con_lda)
df_participacion_2024

,ID_PARTICIPACION,PARLAMENTARIO_ID,FECHA,LEGISLATURA,SESION,TIPO_SESION,TIPO_PARTICIPACION,CAMARA,PARLAMENTARIOS,TEXTO_PRINCIPAL,TEXTO_ANTECEDENTE,DESCRIPCION_DEBATE,BOLETIN_ID,KEYWORDS
0,0,3494,2024-09-03,372,51,Sesión parlamentaria ordinaria,Intervención,Senado,[3494],"La señora ALLENDE.- \nGracias, Presidente. \nH...",El señor GARCÍA ( Presidente ).- \nEn el prime...,ACUERDO ENTRE CHILE Y ESTADOS UNIDOS DE AMÉRIC...,NaN,"productos, presidente, europea, unidos, acuerdo"
1,1,3494,2024-09-03,372,51,Sesión parlamentaria ordinaria,Intervención,Senado,[3494],"La señora ALLENDE.- \nGracias, Presidente. \nC...",El señor GARCÍA ( Presidente ).- \nProyecto de...,REGLAS DE IGUALDAD DE GÉNERO PARA DESIGNACIÓN ...,NaN,"abriendo, manejo, proyecto, pesca, mujeres"
2,2,3494,2024-09-03,372,51,Sesión parlamentaria ordinaria,Intervención,Senado,[3494],"La señora ALLENDE.- \nGracias, Presidente. \nP...","El señor GARCÍA ( Presidente ).- \nProyecto, i...",IMPOSIBILIDAD DE ADJUDICACIÓN DE PROGRAMAS O I...,NaN,"pensiones, deudores, bastante, registro, presi..."
3,3,3494,2024-09-03,372,51,Sesión parlamentaria ordinaria,Intervención,Senado,[3494],"La señora ALLENDE.- \nPresidente , yo le pedí ...",El señor GARCÍA ( Presidente ).- \nEn el prime...,ACUERDO ENTRE CHILE Y ESTADOS UNIDOS DE AMÉRIC...,NaN,"moción, ministerio, ministro, perecibles, tran..."
4,4,3494,2024-08-28,372,48,Sesión parlamentaria extraordinaria,Intervención,Senado,[3494],"La señora ALLENDE.- \nPresidente, como ha seña...",El señor GARCÍA ( Presidente ).- \nProyecto de...,MODIFICACIÓN DE CÓDIGO DE AGUAS EN LO RELATIVO...,NaN,"infractor, menores, infracciones, simplificado..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21866,242897,4622,2024-01-09,371,128,Sesión parlamentaria ordinaria,Solicitud de Resolución,Cámara de Diputados,"[5239, 5261, 5219, 5190, 4602, 4551, 4622, 524...",El señor CIFUENTES (Presidente).- \n \n Corres...,NaN,PREOCUPACIÓN POR AFECTACIÓN A PATRIMONIO MUNIC...,NaN,"prosecretario, resolución, nel, señor, solicitud"
21867,242898,4622,2024-01-08,371,128,Sesión parlamentaria ordinaria,Intervención en Comisión,Cámara de Diputados,[4622],La Diputada Yeomans (Presidenta) reconoció la ...,Tratado y acordado en la sesión especial celeb...,﻿INFORME DE LA COMISION DE HACIENDA RECAÍDO EN...,16498.0,"regular, respecto, sector, trabajan, yeomans"
21868,242899,4622,2024-01-03,371,124,Sesión parlamentaria ordinaria,Petición de oficio,Cámara de Diputados,[4622],"-DiputadaYeomans , doña Gael . Acciones dest...",NaN,OTROS DOCUMENTOS DE LA CUENTA,NaN,"santiago, secretaría, suspensión, tras, victoria"
21869,242900,4622,2024-01-03,371,124,Sesión parlamentaria ordinaria,Petición de oficio,Cámara de Diputados,[4622],"-DiputadaYeomans , doña Gael . Acciones dest...",NaN,OTROS DOCUMENTOS DE LA CUENTA,NaN,"sandra, servicio, suspensión, tras, victoria"
